<a href="https://colab.research.google.com/github/Taichi2005/GoogleColab_Whisper/blob/main/%E9%AB%98%E6%80%A7%E8%83%BD%E6%96%87%E5%AD%97%E8%B5%B7%E3%81%93%E3%81%97%E5%AE%9F%E8%A1%8C%E3%82%BB%E3%83%AB_(large_v3_turbo%E5%AF%BE%E5%BF%9C%E7%89%88).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# セル1: 環境構築
# -------------------------------------------------------------------------------------------
# GPUが有効になっているかを確認
print("▼ GPUの確認")
!nvidia-smi

# faster-whisperと、GPUで動作させるために必要なCUDAライブラリをインストール
print("\n▼ 必要なライブラリのインストール")
!pip install faster-whisper==1.0.3 -q
!pip install nvidia-cublas-cu12==12.6.4.1 -q
!pip install nvidia-cudnn-cu12==9.10.2.21 -q

print("\n✅ 環境構築が完了しました。")

In [ ]:
# -------------------------------------------------------------------------------------------
# セル2: Google Driveへの接続
# -------------------------------------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive')
print("\n✅ Google Driveへの接続が完了しました。")

In [ ]:
#### **【修正・完成版】セル 3: 文字起こしの設定と実行**


#@title 🚀 高性能文字起こし実行セル (large-v3-turbo対応版)
#@markdown ### 1. Google Driveのパス設定
#@markdown ---
#@markdown 音声ファイルが保存されているフォルダと、結果を保存するフォルダのパスを指定します。
drive_audio_input_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/input_audio" #@param {type:"string"}
drive_transcript_output_dir = "/content/drive/MyDrive/Colab/Whisper_Transcripts/output_transcripts" #@param {type:"string"}

#@markdown ### 2. モデルとパフォーマンス設定
#@markdown ---
#@markdown **モデル**: `deepdml/faster-whisper-large-v3-turbo-ct2`は、`large-v3-turbo`をfaster-whisper用に変換した高速モデルです。<br>
#@markdown **計算タイプ**: `float16`は、精度をほぼ維持したままVRAM使用量を半減させ、高速化する推奨設定です。
model_name = "deepdml/faster-whisper-large-v3-turbo-ct2" #@param ["deepdml/faster-whisper-large-v3-turbo-ct2", "large-v3", "large-v2", "distil-large-v3", "medium"]
compute_type = "float16" #@param ["float16", "int8"]

#@markdown ### 3. VAD (音声区間検出) 設定
#@markdown ---
#@markdown VADを有効にすると、無音区間を自動で除去し、文字起こしの精度を向上させることができます。
use_vad_filter = True #@param {type:"boolean"}
#@markdown ここで指定したミリ秒以上の無音を「発話の区切り」とみなします。
vad_min_silence_duration_ms = 500 #@param {type:"slider", min:100, max:2000, step:100}

# --- ここから下は実行コード（編集不要） ---
import os
import glob
import shutil
from faster_whisper import WhisperModel
import time

# 1. 環境設定とディレクトリの準備
print("--- 1. 環境設定とディレクトリの準備 ---")
os.makedirs(drive_audio_input_dir, exist_ok=True)
os.makedirs(drive_transcript_output_dir, exist_ok=True)
print(f"入力フォルダ: {drive_audio_input_dir}")
print(f"出力フォルダ: {drive_transcript_output_dir}")

# 2. モデルのロード
print(f"\n--- 2. モデル '{model_name}' ({compute_type}) をロード中... ---")
start_load_time = time.time()
# model_size_or_path パラメータにHugging Faceのリポジトリパスを指定することで、
# CTranslate2形式に変換済みのカスタムモデルを直接ダウンロードして使用できます。
model = WhisperModel(model_name, device="cuda", compute_type=compute_type)
end_load_time = time.time()
print(f"✅ モデルのロードが完了しました。({end_load_time - start_load_time:.2f}秒)")

# 3. 処理対象ファイルの検索
print("\n--- 3. 処理対象ファイルの検索 ---")
supported_formats = ['*.mp3', '*.wav', '*.m4a', '*.flac', '*.ogg', '*.opus']
audio_files = []
for fmt in supported_formats:
    audio_files.extend(glob.glob(os.path.join(drive_audio_input_dir, fmt)))

if not audio_files:
    print("⚠️ 入力フォルダに処理対象の音声ファイルが見つかりませんでした。")
else:
    print(f"✅ {len(audio_files)} 件の音声ファイルを検出しました。")
    # 4. メイン処理ループ
    print("\n--- 4. 文字起こし処理開始 ---")
    for i, audio_path_drive in enumerate(audio_files):
        start_process_time = time.time()
        filename = os.path.basename(audio_path_drive)
        print(f"\n[{i+1}/{len(audio_files)}] 処理開始: {filename}")

        try:
            # 戦略: ネットワーク遅延を避けるため、ファイルをColabローカルに一時コピー
            local_audio_path = os.path.join('/content/', filename)
            shutil.copy(audio_path_drive, local_audio_path)
            print(f"  - ファイルをローカルにコピーしました。")

            # 文字起こし実行
            print(f"  - 文字起こしを実行中... (VAD: {'有効' if use_vad_filter else '無効'})")
            segments, info = model.transcribe(
                local_audio_path,
                beam_size=5,
                vad_filter=use_vad_filter,
                vad_parameters=dict(min_silence_duration_ms=vad_min_silence_duration_ms)
            )
            print(f"  - 検出言語: {info.language} (確率: {info.language_probability:.2f})")

            # 結果をタイムスタンプ付きテキストとして保存
            base_filename, _ = os.path.splitext(filename)
            output_txt_filename = f"{base_filename}.txt"
            local_output_path = os.path.join('/content/', output_txt_filename)
            drive_output_path = os.path.join(drive_transcript_output_dir, output_txt_filename)

            with open(local_output_path, "w", encoding="utf-8") as f:
                for segment in segments:
                    line = f"[{segment.start:0>7.2f}s -> {segment.end:0>7.2f}s] {segment.text.strip()}\n"
                    f.write(line)

            # 戦略: 処理結果をGoogle Driveに移動
            shutil.move(local_output_path, drive_output_path)
            print(f"  - 結果を保存しました: {drive_output_path}")

        except Exception as e:
            print(f"  - 💥 エラーが発生しました: {e}")

        finally:
            # 戦略: ローカルの一時ファイルをクリーンアップ
            if 'local_audio_path' in locals() and os.path.exists(local_audio_path):
                os.remove(local_audio_path)
            if 'local_output_path' in locals() and os.path.exists(local_output_path):
                os.remove(local_output_path) # moveに失敗した場合に備える
            end_process_time = time.time()
            print(f"  - 処理完了 ({end_process_time - start_process_time:.2f}秒)")

    print("\n--- 🎉 全てのファイルの処理が完了しました ---")